In [1]:
import bs4 as bs  # BeautifulSoup
import pandas as pd
import urllib.request
import re
import nltk
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
import heapq
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from collections import defaultdict

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zmtyo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zmtyo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def _scrape_webpage_fixed(fixed, url):
    """
    Use BeautifulSoup to scrape the webpage text contents.
    """    
    scraped_textdata = urllib.request.urlopen(url)
    textdata = scraped_textdata.read()
    parsed_textdata = bs.BeautifulSoup(textdata,'lxml')
    #paragraphs = parsed_textdata.find_all(['p', 'dd','li'])  #dd and li includes some informations, but some unrealted
    paragraphs = parsed_textdata.find_all(['p'])
    formated_text = ""
    
    if fixed == 0:
        for para in paragraphs:
            formated_text += para.text
    else:
        for para in paragraphs:
            line = para.text.lower() # change to lower case
            line = re.sub(r"[^a-z\s]", "", line.strip())  # only contains english words
            formated_text += line
    return formated_text

In [3]:
fixed_text = _scrape_webpage_fixed(1 , 'https://en.wikipedia.org/wiki/Natural_language_processing')

#text
fixed_text[0:1000]

'natural language processing nlp is a subfield of linguistics computer science and artificial intelligence concerned with the interactions between computers and human language in particular how to program computers to process and analyze large amounts of natural language datachallenges in natural language processing frequently involve speech recognition natural language understanding and naturallanguage generationnatural language processing has its roots in the s already in  alan turing published an article titled computing machinery and intelligence which proposed what is now called the turing test as a criterion of intelligence a task that involves the automated interpretation and generation of natural language but at the time not articulated as a problem separate from artificial intelligencethe premise of symbolic nlp is wellsummarized by john searles chinese room experiment given a collection of rules eg a chinese phrasebook with questions and matching answers the computer emulates

In [4]:
nonfixed_test = _scrape_webpage_fixed(0 , 'https://en.wikipedia.org/wiki/Natural_language_processing')

#text
nonfixed_test[0:1000]

'Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data.\nChallenges in natural language processing frequently involve speech recognition, natural language understanding, and natural-language generation.\nNatural language processing has its roots in the 1950s. Already in 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence, a task that involves the automated interpretation and generation of natural language, but at the time not articulated as a problem separate from artificial intelligence.\nThe premise of symbolic NLP is well-summarized by John Searle\'s Chinese room experiment: Given a collection of rules (e.g., a Chinese phrasebook, with questions an

## 1.1 (0.3 point) Follow code examples to generate N-grams with NLTK

In [5]:
def get_ngrams(text, N):
    text_token = nltk.word_tokenize(text.lower())
    n_grams = ngrams(text_token, N)
    return [" ".join(grams) for grams in n_grams]

n_gram_res = get_ngrams(nonfixed_test, 3)
n_gram_res

['natural language processing',
 'language processing (',
 'processing ( nlp',
 '( nlp )',
 'nlp ) is',
 ') is a',
 'is a subfield',
 'a subfield of',
 'subfield of linguistics',
 'of linguistics ,',
 'linguistics , computer',
 ', computer science',
 'computer science ,',
 'science , and',
 ', and artificial',
 'and artificial intelligence',
 'artificial intelligence concerned',
 'intelligence concerned with',
 'concerned with the',
 'with the interactions',
 'the interactions between',
 'interactions between computers',
 'between computers and',
 'computers and human',
 'and human language',
 'human language ,',
 'language , in',
 ', in particular',
 'in particular how',
 'particular how to',
 'how to program',
 'to program computers',
 'program computers to',
 'computers to process',
 'to process and',
 'process and analyze',
 'and analyze large',
 'analyze large amounts',
 'large amounts of',
 'amounts of natural',
 'of natural language',
 'natural language data',
 'language data .'

## 1.2 (2 points) Write the code for text summarization with N-grams

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
nonfixed_sentance = nonfixed_test.split('.')
nonfixed_sentance[0:2]

['Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data',
 '\nChallenges in natural language processing frequently involve speech recognition, natural language understanding, and natural-language generation']

In [8]:
#from string import punctuation
#from nltk.corpus import stopwords
stopword = stopwords.words('english')
from string import punctuation
def cleanUpSentense(token_sent):
    senttoken = nltk.word_tokenize(token_sent)
    #leSent = lemmatizeWords(senttoken)  # lemmatize senstnce
    noStopwords = [word for word in senttoken if word not in stopword]  # remove stop words
    formated_text = ""
    
    for para in noStopwords:
        line = para.lower() # change to lower case
        line = line.replace('\n','')
        outNumline = ''.join(c for c in line if not c.isdigit())  # take off number
        outPunc = ''.join(c for c in outNumline if c not in punctuation) # take off puncctutation
        formated_text += outPunc + " "
    return formated_text

#print(cleanUpSentense(token_sent[0]))

In [9]:
for idx in range(0,len(nonfixed_sentance)):
    nonfixed_sentance[idx] = cleanUpSentense(nonfixed_sentance[idx])
nonfixed_sentance

['natural language processing  nlp  subfield linguistics  computer science  artificial intelligence concerned interactions computers human language  particular program computers process analyze large amounts natural language data ',
 'challenges natural language processing frequently involve speech recognition  natural language understanding  naturallanguage generation ',
 'natural language processing roots s ',
 'already   alan turing published article titled  computing machinery intelligence  proposed called turing test criterion intelligence  task involves automated interpretation generation natural language  time articulated problem separate artificial intelligence ',
 'the premise symbolic nlp wellsummarized john searle s chinese room experiment  given collection rules  e ',
 'g ',
 ' chinese phrasebook  questions matching answers   computer emulates natural language understanding  nlp tasks  applying rules data confronted ',
 'up s  natural language processing systems based compl

In [10]:
cleaned_text = cleanUpSentense(nonfixed_test)

In [11]:
fixed_sentance = nonfixed_sentance

In [12]:
n_gram_res = get_ngrams(cleaned_text, 3)
three_gram_set = set(n_gram_res)
three_gram_set

{'a coarse division',
 'a major drawback',
 'abandoned statistical methods',
 'achieve stateoftheart results',
 'acquiring knowledge understanding',
 'action process acquiring',
 'additional information this',
 'advantage express relative',
 'advantages handproduced rules',
 'aid solving larger',
 'alan turing published',
 'algorithm alike without',
 'algorithms applied naturallanguageprocessing',
 'algorithms assign relative',
 'algorithms decision trees',
 'algorithms language processing',
 'algorithms many advantages',
 'algorithms perspective cognitive',
 'algorithms presented us',
 'algorithms take input',
 'alignment language modeling',
 'alike without additional',
 'along findings cognitive',
 'along sequence n',
 'already alan turing',
 'ambiguous person cognitive',
 'amounts natural language',
 'analysis large corpora',
 'analyze large amounts',
 'analyzed pmm probable',
 'analyzed the mathematical',
 'annotations typical realworld',
 'another ” provides',
 'answering instead 

In [13]:
# Getting trigrams  
ngram_collection = CountVectorizer(ngram_range = (3,3)) 
X = ngram_collection.fit_transform(fixed_sentance)  
features = (ngram_collection.get_feature_names()) 
  
# Applying TFIDF 
vectorizer = TfidfVectorizer(ngram_range = (3,3)) 
Y = vectorizer.fit_transform(fixed_sentance) 
#scores = (Y.toarray()) 
  
# Getting scores 
sums = Y.sum(axis = 0) 
data = [] 
for col, term in enumerate(features): 
    data.append( (term, sums[0,col] )) 

# transfer score data from list to dict
three_grams_score_dict = defaultdict(list)
for k, v in data:
    three_grams_score_dict[k].append(v)
three_grams_score_dict

defaultdict(list,
            {'abandoned statistical methods': [0.30151134457776363],
             'achieve stateoftheart results': [0.1816193761464213],
             'acquiring knowledge understanding': [0.3333333333333333],
             'action process acquiring': [0.3333333333333333],
             'advantage express relative': [0.23570226039551578],
             'advantages handproduced rules': [0.1732494523866694],
             'aid solving larger': [0.28867513459481287],
             'alan turing published': [0.19611613513818396],
             'algorithm alike without': [0.24253562503633297],
             'algorithms applied naturallanguageprocessing': [0.408248290463863],
             'algorithms assign relative': [0.18424559208156652],
             'algorithms decision trees': [0.26900202866756945],
             'algorithms language processing': [0.3141045009309007],
             'algorithms many advantages': [0.1732494523866694],
             'algorithms perspective cognitive'

In [14]:
sentScore = dict()

for sent in fixed_sentance:
    sentScore[sent] = 0;


In [15]:
for sent in fixed_sentance:
    for words in get_ngrams(sent, 3):
        if words in three_grams_score_dict.keys():
            sentScore[sent] += three_grams_score_dict[words][0]
dict(list(sentScore.items())[0:3])

{'natural language processing  nlp  subfield linguistics  computer science  artificial intelligence concerned interactions computers human language  particular program computers process analyze large amounts natural language data ': 6.9479480543941765,
 'challenges natural language processing frequently involve speech recognition  natural language understanding  naturallanguage generation ': 5.4951195940727064,
 'natural language processing roots s ': 3.011973479322823}

In [16]:
summary_sentence = heapq.nlargest(3,sentScore, key = sentScore.get)
summary = ' '.join(summary_sentence)
summary

'however  partofspeech tagging introduced use hidden markov models natural language processing  increasingly  research focused statistical models  make soft  probabilistic decisions based attaching realvalued weights features making input data     george lakoff offers methodology build natural language processing  nlp  algorithms perspective cognitive science  along findings cognitive linguistics     the first defining aspect cognitive task nlp application theory conceptual metaphor  explained lakoff “ understanding one idea  terms another ” provides idea intent author  more recent systems based machinelearning algorithms many advantages handproduced rules  despite popularity machine learning nlp research  symbolic methods still    commonly used since socalled  statistical revolution        late s mids  much natural language processing research relied heavily machine learning '

## 1.2 (0.7 point) Compare with Lab 1 methods. Please clearly explain

In lab 1, we only care about scores for one word. Even we take of unrelated words, the score of the word could be not accurate when back to a sentence since we may hard to figure out some combinations. For example, 'natural' is having a high score, but the word 'natural' combine with different words shall have a different scores. In our content, if it combines with 'natural language + something' the score shall be higher, but it is 'xxx in natural'， it shall get a low score since our most important combination shall be natural language. 
In this lab, with n-gram, it fixed the problem. It don't need be supported by a large size score backup, it only needs couple sentances and able to get more acurate feedback. With larger n, the model can have well-understood to the content and enabling small experiments to scale up efficiently. This is the reason most important sentance will have high score because of a combination instead of one word.
An example will be (not in the lab content):
if score 'ha' is having a fine score. What if there is a part with 'hahahaha.......*100', at this point, this sentance will contain a extrem high score. But this sentance cloud be the one not related.